# P3 Avistamiento de aves

## Obtener datos (scraping)

Primero vamos a obtener los datos. Para ello nos visitaremos las web de avistamientos de aves [shorebirder](https://www.shorebirder.com/), [trevorsbirding](https://www.trevorsbirding.com/) y [dantallmansbirdblog](https://dantallmansbirdblog.blogspot.com/).

Durante la visita a la web y haciendo uso del inspector (F12) podemos ver que las descripciones que necesitamos se encuentran en los tag de párrafo (entre *\<p\> TEXTO \</p\>*). Sabiendo eso vamos a crear funciones de utilidad que se encargarán de descargar el contenido de la web y extraer el texto.

Las descargas las realizaremos en `data/raw` mientras que en `data/posts` guardaremos los textos encontrados.

** `dantallmansbirdblog` tiene una estructura ligeramente diferente (entre *\<p\>\</p\> TEXTO \<p\>\</p\>*), a lo que tendremos que modificar la función `get_texts` (a continuación) para obtener sus textos.

In [ ]:
import os
import re
import wget

data_raw_path = "../data/raw"
data_posts_path = "../data/posts"

def maybe_mkdir(path):
  try:
    os.mkdir(path)
  except OSError as error:
    print(error)

maybe_mkdir(data_raw_path)
maybe_mkdir(data_posts_path)

def download(url, out_label):
  return wget.download(url, out=f"{data_raw_path}/{out_label}")

def get_texts(filename):
  file = open(filename, 'r')
  text = file.read()

  # get texts
  get_p = re.compile(r'<p>((.|\n)*?)</p>')
  texts = get_p.findall(text)

  # remove styling and inner tags
  remove_tags = re.compile(r'(<.*?>)|\\n| +(?= )|\\|\&.+?\;')
  texts = map(lambda text: re.sub(remove_tags, "", str(text[0])), texts)
  texts = list(texts)

In [ ]:
import re

def get_texts(filename):
  file = open(filename, 'r')
  text = file.read()

  # get texts
  get_p = re.compile(r'<p>((.|\n)*?)</p>')
  texts = get_p.findall(text)

  # remove styling and inner tags
  remove_tags = re.compile(r'(<.*?>)|\\n| +(?= )|\\|\&.+?\;')
  texts = map(lambda text: re.sub(remove_tags, "", str(text[0])), texts)
  texts = list(texts)


shorebirder_home = download("https://www.shorebirder.com/", "shorebirder_home.html")
posts = get_texts(shorebirder_home)
shorebirder_posts_file = open(f"{data_posts_path}/{shorebirder_home}.txt", "w")
shorebirder_posts_file.writelines(posts)

## Analizar articulos

In [ ]:
# Cargar dependencias para el nlp
import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm
nlp = en_core_web_sm.load()

In [ ]:
# review 320 de usa_nevada_las-vegas_excalibur_hotel_casino
text = "fantastic value for money at the excalibur	My husband and i stayed at the excalibur from the 6th to 16th april we had a wonderful time, this hotel is in a great spot on strip, we were in tower 2 and the room was spotless, cleaned every day, food was great value from buffet bar, staff were very friendly and helpful, if you are going to visit gran canyon book with sundance they are a class act just a fantastic place to visit las vegas cant wait to go back"

train = nlp(text)
displacy.render(train, jupyter=True, style="ent")